In [3]:
import torch
import torch.nn as nn
import flwr as fl
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Client device: {device}")

file_path = "framingham_part1.csv"
data = pd.read_csv(file_path)


scaler = StandardScaler()
X_client = scaler.fit_transform(data.drop(columns=['TenYearCHD']).values)
y_client = data['TenYearCHD'].values


def add_noise(data, noise_level=0.01):
    noise = np.random.normal(0, noise_level, data.shape)
    return data + noise

X_client = add_noise(X_client, noise_level=0.01)


train_dataset = TensorDataset(torch.tensor(X_client, dtype=torch.float32).to(device),
                              torch.tensor(y_client, dtype=torch.float32).to(device))
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# ✅ Define Local Model with Improved Generalization
class LocalModel(nn.Module):
    def __init__(self, input_size):
        super(LocalModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

input_size = X_client.shape[1]
model = LocalModel(input_size).to(device)

# Define Differential Privacy with Gradient Clipping
class FedProxLoss(nn.Module):
    def __init__(self, mu=0.01):
        super(FedProxLoss, self).__init__()
        self.mu = mu

    def forward(self, preds, labels, local_params, global_params):
        base_loss = nn.BCELoss()(preds, labels)
        prox_loss = sum((torch.norm(local_param - global_param) ** 2).sum()
                        for local_param, global_param in zip(local_params, global_params))
        return base_loss + (self.mu / 2) * prox_loss

optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)  # ✅ Reduced learning rate & added weight decay
scheduler = StepLR(optimizer, step_size=10, gamma=0.85)
fedprox_loss = FedProxLoss(mu=0.01)

#  Secure Aggregation and Differential Privacy
def clip_gradients(model, max_norm=0.5):  
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)

#  Flower Client with Secure Aggregation 
class FLClient(fl.client.NumPyClient):
    def __init__(self, model, train_loader):
        self.model = model
        self.train_loader = train_loader
        self.global_params = None

    def get_parameters(self, config):
        return [val.cpu().detach().numpy() for val in self.model.parameters()]

    def set_parameters(self, parameters):
        self.global_params = [torch.tensor(p).to(device) for p in parameters]
        state_dict = self.model.state_dict()
        for name, param in zip(state_dict.keys(), self.global_params):
            state_dict[name] = param
        self.model.load_state_dict(state_dict)
        print("✅ Client: Parameters received & updated.")

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        self.model.train()

        total_loss = 0  # Track total loss
        correct, total = 0, 0

        for epoch in range(10): 
            for X_batch, y_batch in self.train_loader:
                optimizer.zero_grad()
                y_pred = self.model(X_batch).squeeze()
                loss = fedprox_loss(y_pred, y_batch, list(self.model.parameters()), self.global_params)
                loss.backward()
                clip_gradients(self.model, max_norm=0.5)  
                optimizer.step()
                
                total_loss += loss.item()  
                correct += ((y_pred > 0.5) == y_batch).sum().item()
                total += y_batch.size(0)

        scheduler.step()
        avg_loss = total_loss / len(self.train_loader)  # Compute average loss
        client_accuracy = correct / total
        print(f"📌 Client: Training Completed | Accuracy: {client_accuracy:.4f} | Loss: {avg_loss:.4f}")
        return self.get_parameters(config), total, {"accuracy": client_accuracy, "loss": avg_loss}  # ✅ Include loss

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        self.model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for X_batch, y_batch in self.train_loader:
                y_pred = self.model(X_batch).squeeze()
                correct += ((y_pred > 0.5) == y_batch).sum().item()
                total += y_batch.size(0)
        val_accuracy = correct / total
        print(f"📌 Client: Validation Accuracy: {val_accuracy:.4f}")
        return 0.0, total, {"accuracy": val_accuracy}

# Connect to Server with Secure Aggregation
print("🚀 Client: Connecting to the global server...")
fl.client.start_client(
    server_address="localhost:8080",
    client=FLClient(model, train_loader).to_client()
)


✅ Client device: cuda


	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      
INFO :      Received: get_parameters message bc2ddb76-b8fc-48dc-a26a-8d54e6e7eb92
INFO :      Sent reply


🚀 Client: Connecting to the global server...


INFO :      
INFO :      Received: train message d8b777ec-0892-4f2d-862e-e7c93f9ec936


✅ Client: Parameters received & updated.


...

INFO :      
INFO :      Received: train message db41524e-5619-47dd-be36-90751bcc46fd


✅ Client: Parameters received & updated.


INFO :      Sent reply
INFO :      
INFO :      Received: evaluate message a00fc825-5e76-4277-8ef2-abd223a41c17
INFO :      Sent reply


📌 Client: Training Completed | Accuracy: 0.8707 | Loss: 3.1564
✅ Client: Parameters received & updated.
📌 Client: Validation Accuracy: 0.8698


INFO :      
INFO :      Received: reconnect message 53521896-0163-4cf3-9332-211c70c7792c
INFO :      Disconnect and shut down
